In [39]:
import os
from bs4 import BeautifulSoup
from datetime import datetime
import shutil
import requests
import time

In [27]:
agenda_path = "scraping/agenda"
agenda_fnames = os.listdir(agenda_path)
minutes_path = "scraping/minutes"
minutes_fnames = os.listdir(agenda_path)

minutes_path_new = "scraping/minutes_new"


In [4]:
f = open(f"{agenda_path}/{agenda_fnames[-20]}", "r")

In [28]:
def get_fname_2(date_str, meeting_type):
    """
    date_str like: "2022-10-03",
    meeting_type like: "Accessibility Advisory Committee Meeting"
    """
    meeting_type = meeting_type.replace(" ", "-")
    meeting_type = meeting_type.replace(",","")
    return f"{date_str}.{meeting_type}"

In [43]:
1288+115

1403

In [46]:
abc = "01-11-21 G"

In [48]:
abc.split(" ")

['01-11-21', 'G']

In [ ]:
if " " in abc:
    possible_short_date = abc.split(" ")[0]
    try:
        date = datetime.strptime(possible_date, "%m-%d-%y")
    except:
        continue

In [51]:
agenda_fnames[800]

'2016-02-01.City-Council-Meeting.html'

In [55]:
text = "April 7 2015 - Draft Minutes 04-07-2015 - April 7 2015 ccm.pdf".lower()

In [56]:
text

'april 7 2015 - draft minutes 04-07-2015 - april 7 2015 ccm.pdf'

In [61]:
possible_date = text.split(" - ")[1].split(" ")[-1]

In [63]:
date = datetime.strptime(possible_date, "%m-%d-%Y")

datetime.datetime(2015, 4, 7, 0, 0)

In [65]:
def try_again(a):
    text = a.text.lower()
    if "minutes" not in text:
        return
    possible_date = text.split(" - ")[1].split(" ")[-1]
    date = datetime.strptime(possible_date, "%m-%d-%Y")
    return date


In [67]:
agenda_fname

'2015-04-27.City-Council-Meeting.html'

In [76]:
agenda_fname = agenda_fnames[703]
meeting_type_str = agenda_fname.split(".")[1]
f = open(f"{agenda_path}/{agenda_fname}", "r")
agenda_text = f.read()
f.close()
soup = BeautifulSoup(agenda_text, "html.parser")
attachment_divs = soup.find_all("div", attrs={"class": "AgendaItemAttachment"})

In [77]:
attachment_divs

[<div class="panel-body AgendaItemAttachment AgendaItemAttachmentNotSelected">No Item Selected</div>,
 <div class="AgendaItemAttachment AgendaItemAttachmentNone">This item has no attachments</div>,
 <div class="AgendaItemAttachment AgendaItemAttachment24049"><span class="Sequence">1.</span><a class="Link" data-container="body" data-html="true" data-original-title="April 1 2015 - Draft Minutes 04-01-2015 - April 1 2015 AAC.pdf" data-toggle="tooltip" href="filestream.ashx?DocumentId=6633" target="_blank"><span class="Link">April 1 2015 - Draft Minutes 04-01-2015 - April 1 2015 AAC.pdf</span></a></div>,
 <div class="AgendaItemAttachment AgendaItemAttachment24050"><span class="Sequence">1.</span><a class="Link" data-container="body" data-html="true" data-original-title="Accessibility Coordinator  Report AAC15-017  Prese - AAC15-017 Presentation about Live Streaming Options for the Council Chambers.pdf" data-toggle="tooltip" href="filestream.ashx?DocumentId=6634" target="_blank"><span class

In [90]:
for n, agenda_fname in enumerate(agenda_fnames[1252:]):
    print(n, end=", ")
    meeting_type_str = agenda_fname.split(".")[1]
    f = open(f"{agenda_path}/{agenda_fname}", "r")
    agenda_text = f.read()
    f.close()
    soup = BeautifulSoup(agenda_text, "html.parser")
    attachment_divs = soup.find_all("div", attrs={"class": "AgendaItemAttachment"})
    for attachment in attachment_divs:
        a = attachment.find("a")
        if not a:
            continue
        href = a.attrs["href"]
        url = f"https://pub-peterborough.escribemeetings.com/{href}"
        file_id = url.split("=")[-1]
        old_pdf_name = f"{file_id}.pdf"


        if "minutes" in a.text.lower():
            possible_date = text.split(" - ")[1].split(" ")[-1]
            date = datetime.strptime(possible_date, "%m-%d-%Y")
        else:
            # continue
            possible_date = a.text[:10]
            if possible_date.count("-") != 2:
                continue

            if " " in possible_date:
                possible_short_date = possible_date.split(" ")[0]
                try:
                    date = datetime.strptime(possible_short_date, "%m-%d-%y")
                except:
                    continue
            else:
                try:
                    date = datetime.strptime(possible_date, "%m-%d-%Y")
                except ValueError:
                    try:
                        date = datetime.strptime(possible_date, "%Y-%m-%d")
                    except:
                        continue

        date_str = datetime.strftime(date, "%Y-%m-%d")
            
        new_fname = f"{date_str}.{meeting_type_str}.pdf"
        new_fpath = f"{minutes_path_new}/{new_fname}"

        # do not overwrite
        if os.path.isfile(new_fpath):
            continue

        if old_pdf_name in minutes_fnames:
            print(f"copying {old_pdf_name} to {new_fpath}")
            # copy
            shutil.copy(f"{minutes_path}/{old_pdf_name}", new_fpath)
        else:
            # download
            resp = requests.get(url)

            with open(new_fpath, "wb") as f:
                f.write(resp.content)
            print(f"download and save {new_fpath}")

            time.sleep(2)

0, 1, 2, 3, 4, 5, 6, 7, download and save scraping/minutes_new/2019-09-05.Peterborough-Architectural-Conservation-Advisory-Committee.pdf
8, download and save scraping/minutes_new/2019-10-15.Finance-Committee-Meeting.pdf
9, 10, download and save scraping/minutes_new/2019-10-10.Arts-Culture-and-Heritage-Advisory-Committee-Meeting.pdf
11, download and save scraping/minutes_new/2019-10-10.Museum-and-Archives-Advisory-Committee-Meeting.pdf
12, download and save scraping/minutes_new/2019-10-16.Peterborough-Environmental-Advisory-Committee.pdf
13, download and save scraping/minutes_new/2019-10-28.City-Council-Meeting.pdf
download and save scraping/minutes_new/2019-11-04.City-Council-Meeting.pdf
14, download and save scraping/minutes_new/2019-10-22.Arenas-Parks-and-Recreation-Advisory-Committee-Meeting.pdf
15, download and save scraping/minutes_new/2019-10-24.Joint-Services-Steering-Committee-Meeting.pdf
16, download and save scraping/minutes_new/2019-11-04.General-Committee.pdf
download and s

In [84]:
a

In [82]:
attachment_divs[2]

<div class="AgendaItemAttachment AgendaItemAttachment24049"><span class="Sequence">1.</span><a class="Link" data-container="body" data-html="true" data-original-title="April 1 2015 - Draft Minutes 04-01-2015 - April 1 2015 AAC.pdf" data-toggle="tooltip" href="filestream.ashx?DocumentId=6633" target="_blank"><span class="Link">April 1 2015 - Draft Minutes 04-01-2015 - April 1 2015 AAC.pdf</span></a></div>

In [81]:
attachment

<div class="AgendaItemAttachment AgendaItemAttachment24056"><span class="Sequence">3.</span><a class="Link" data-container="body" data-html="true" data-original-title="Accessibility Compliance Coordinator  Report AAC15 - AAC15-023 Appendix B - APS Inventory.pdf" data-toggle="tooltip" href="filestream.ashx?DocumentId=6642" target="_blank"><span class="Link">Accessibility Compliance Coordinator  Report AAC15 - AAC15-023 Appendix B - APS Inventory.pdf</span></a></div>

In [80]:
possible_date

'04-07-2015'

In [79]:
date

datetime.datetime(2015, 4, 7, 0, 0)

In [54]:
attachment_divs

[<div class="panel-body AgendaItemAttachment AgendaItemAttachmentNotSelected">No Item Selected</div>,
 <div class="AgendaItemAttachment AgendaItemAttachmentNone">This item has no attachments</div>,
 <div class="AgendaItemAttachment AgendaItemAttachment23949"><span class="Sequence">1.</span><a class="Link" data-container="body" data-html="true" data-original-title="Planning Committee Report No. 5   Planning Committ - Report 5 of Planning Committee April 13 2015.pdf" data-toggle="tooltip" href="filestream.ashx?DocumentId=11374" target="_blank"><span class="Link">Planning Committee Report No. 5   Planning Committ - Report 5 of Planning Committee April 13 2015.pdf</span></a></div>,
 <div class="AgendaItemAttachment AgendaItemAttachment23950"><span class="Sequence">1.</span><a class="Link" data-container="body" data-html="true" data-original-title="Committee of the Whole Report No. 7  Committee of  - Report 7 Committee of Whole Report of April 20 2015.pdf" data-toggle="tooltip" href="filest

In [45]:
url

'https://pub-peterborough.escribemeetings.com/filestream.ashx?DocumentId=28523'

In [32]:
a.text

'June 30 2008 - Draft Minutes 06-30-2008 - June 30 2008 cwm.pdf'

In [ ]:
for attachment in attachment_divs:
    attachment.find("a")

In [7]:
a = attachment_divs[0].find("a")

In [10]:
a

<a class="Link" data-container="body" data-html="true" data-original-title="07-04-2022 - GC.pdf" data-toggle="tooltip" href="filestream.ashx?DocumentId=34500" tabindex="0" target="_blank"><span class="Link">07-04-2022 - GC.pdf</span></a>

In [13]:
href = a.attrs["href"]

In [26]:
possible_date = a.text[:10]
if possible_date.count("-") != 2:
    # return

2

In [22]:

date = datetime.strptime(possible_date, "%m-%d-%Y")
date_str = datetime.strftime(date, "%Y-%m-%d")

In [25]:
date_iso

'2022-07-04'

In [ ]:
agenda_fnames[0:10]

['2008-07-28.Planning-Committee.html',
 '2008-08-04.Committee-of-the-Whole.html',
 '2008-09-02.Planning-Committee.html',
 '2009-02-09.City-Council-Meeting.html',
 '2009-02-09.Committee-of-the-Whole.html',
 '2009-02-09.Planning-Committee.html',
 '2009-02-17.Planning-Committee.html',
 '2009-02-23.Committee-of-the-Whole.html',
 '2009-03-02.City-Council-Meeting.html',
 '2009-03-02.Committee-of-the-Whole.html']

In [5]:
import json

In [8]:
all_data_flat = json.load(open("scraping/generated_data/all_data_flat.json", "r"))

In [7]:
len(abc)

1640

In [9]:
import pandas as pd

In [91]:
import json

In [92]:
with open("scraping/generated_data/all_meeting_data.json", "r") as f:
    data = json.load(f)

In [96]:
def get_fname(datetime_iso, meeting_type):
    """
    datetime_iso like: "2022-10-03T16:00:00" ",
    meeting_type like: "Accessibility Advisory Committee Meeting"
    """
    date = datetime_iso.split("T")[0]
    meeting_type = meeting_type.replace(" ", "-")
    meeting_type = meeting_type.replace(",","")
    return f"{date}.{meeting_type}"

In [98]:
for d in data:
    fname = f'{get_fname(d["datetime_iso"], d["meeting_type"])}.pdf'
    minutes_fpath = f"{minutes_path_new}/{fname}"
    if os.path.isfile(minutes_fpath):
        d["minutes_filename"] = fname
    else:
        d["minutes_filename"] = ""

In [101]:
json.dump(data, open("scraping/generated_data/all_meeting_data_2.json", "w"))